In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np

### Считаем очищенные данные для первого отделения с добавленными фичами

In [2]:
data = pd.read_csv("final_for_1.csv")

In [3]:
data["half_a_year"] = 0
data["Quarter"] = 0

In [4]:
data.half_a_year[data.Month<7]=1
data.half_a_year[data.Month>6]=2


In [5]:
data.Quarter[data.Month]=1
data.Quarter[data.Month>4]=2
data.Quarter[data.Month>6]=3
data.Quarter[data.Month>9]=4

### Подсчитаем сколько было посещений в полугодии и присвоим данные посещения следующему. Для 2 полугодия 2013 года возьмем средннюю цифру, посмотрев насколько изменяется 1 полугодие и второе

In [6]:
k=0
polugod2 = np.zeros((1,3))
for i in range(2013,2016):
    x = 0
    x = sum(data.Cnt[data.Year==i][data.Month>6])
    polugod2[0,k]=x
    k+=1
k=0
polugod1 = np.zeros((1,2))
for i in range(2014,2016):
    x = 0
    x = x+sum(data.Cnt[data.Year==i][data.Month<7])
    polugod1[0,k]=x
    k+=1
data.number_for_previous_6m[(data.Year==2013) & (data.Month>6)]=46000
for i in range(0,2):    
    data.number_for_previous_6m[(data.Year==2014+i) & (data.Month>6)]=polugod1[0,i]
for i in range(0,2):    
    data.number_for_previous_6m[(data.Year==2014+i) & (data.Month<7)]=polugod2[0,i]


In [7]:
print (" Для первого полугодия 2016 цифра будет", polugod2[0,2])

 Для первого полугодия 2016 цифра будет 25437.0


### Проверим модель на 2 полугодии 2015 года

In [8]:
test = data[data.Year==2015][data.Month>6]
train = data.drop(test.index)

/home/ven/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [9]:
x_test = test.ix[:,1:9]
y_test = test.ix[:,0]
x_train = train.ix[:,1:9]
y_train = train.ix[:,0]
xgtrain = xgb.DMatrix(x_train, label=y_train)
xgtest = xgb.DMatrix(x_test, label=y_test)

In [10]:
watchlist = [(xgtrain, 'train'), (xgtest, 'eval')]
RANDOM_STATE = 42
params = {
            'min_child_weight': 1,
            'eta': 0.001,
            'colsample_bytree': 0.7,
            'max_depth': 13,
            'subsample': 0.7,
            'alpha': 5,
            'gamma': 1,
            'silent': 1,
            'verbose_eval': True,
            'seed': RANDOM_STATE,
            "print.every.n": 100,
            'objective': 'reg:linear'
    
        }

In [11]:
 model = xgb.train(params, xgtrain, 100000, watchlist, early_stopping_rounds=300,maximize=False,verbose_eval = 500)    

[0]	train-rmse:25.4274	eval-rmse:21.0092
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 300 rounds.
[500]	train-rmse:16.8213	eval-rmse:13.732
[1000]	train-rmse:11.7336	eval-rmse:9.97658
[1500]	train-rmse:8.73516	eval-rmse:8.24591
[2000]	train-rmse:6.96307	eval-rmse:7.58071
[2500]	train-rmse:5.92039	eval-rmse:7.38715
[3000]	train-rmse:5.23457	eval-rmse:7.3667
Stopping. Best iteration:
[2851]	train-rmse:5.40966	eval-rmse:7.36354



### eval-rmse:7.36354 для одного часа. Посмотрим относительную ошибку при предсказании нагрузки на все полугодие

In [12]:
tolerance = 100*(sum(model.predict(xgtest)) - polugod2[0,2])/polugod2[0,2]
print ("Относительная ошибка предсказания для полугодия в в целом составляет", tolerance,"%")


Относительная ошибка предсказания для полугодия в в целом составляет 2.74759498567 %


### Обучим модель на полной выборке

In [13]:
x_data = data.ix[:,1:9]
y_data = data.ix[:,0]
xdata = xgb.DMatrix(x_data, label=y_data)

In [14]:
params = {
            'min_child_weight': 1,
            'eta': 0.001,
            'colsample_bytree': 0.7,
            'max_depth': 13,
            'subsample': 0.7,
            'alpha': 5,
            'gamma': 1,
            'silent': 1,
            'verbose_eval': True,
            'seed': RANDOM_STATE,
            "print.every.n": 100,
            'objective': 'reg:linear'
    
        }

In [15]:
model = xgb.cv(params, xdata, 100000, early_stopping_rounds=300,maximize=False,verbose_eval=500)    

[0]	train-rmse:24.7716+0.0986539	test-rmse:24.7711+0.197585
[500]	train-rmse:16.501+0.0588237	test-rmse:16.5716+0.121899
[1000]	train-rmse:11.6272+0.0300698	test-rmse:11.8189+0.0780179
[1500]	train-rmse:8.79017+0.0159055	test-rmse:9.15362+0.0496148
[2000]	train-rmse:7.10118+0.0194012	test-rmse:7.66896+0.039569
[2500]	train-rmse:6.06709+0.0314633	test-rmse:6.84684+0.0329871
[3000]	train-rmse:5.39159+0.0292198	test-rmse:6.37519+0.0186717
[3500]	train-rmse:4.92058+0.0306903	test-rmse:6.08887+0.00348082
[4000]	train-rmse:4.57045+0.0339257	test-rmse:5.90848+0.00783901
[4500]	train-rmse:4.30332+0.0382854	test-rmse:5.79584+0.0197317
[5000]	train-rmse:4.08036+0.0395061	test-rmse:5.71715+0.0275735
[5500]	train-rmse:3.8975+0.0435094	test-rmse:5.66261+0.0338723
[6000]	train-rmse:3.73667+0.0444691	test-rmse:5.62612+0.0428124
[6500]	train-rmse:3.59249+0.046445	test-rmse:5.60053+0.0502463
[7000]	train-rmse:3.46146+0.0443831	test-rmse:5.58194+0.0555173
[7500]	train-rmse:3.3411+0.0469998	test-rmse:5.5

### Точность улучшились. Что связано как с увеличением числа наблюдений, так и с самой методикой оценки. 

In [16]:
model = xgb.train(params, xdata, 9000)    

### Проверим. Количество посещений за 2 полугодие 2015 было 25437.0

In [17]:
x = np.array([[2016,1,11,8,1,25437,1,1],[2016,1,11,9,1,25437,1,1],[2016,1,11,10,1,25437,1,1],[2016,1,11,11,1,25437,1,1]]) # задаем год = 2016, месяц - 1(январь), день месяца 11, час - 10, день недели 
# понедельник - 1 , количество посещений за предыдущее полугодие 25437.0, полугодие -1, квартал - 1

In [18]:
x = pd.DataFrame(x)
x.columns = x_data.columns
proverka = xgb.DMatrix(x)
pr = np.round(model.predict(proverka),0)
print('11 Января 2016 года в 8 утра придет', pr[0], ',в 9 утра', pr[1], ',в 10 утра', pr[2], ',в 11 утра', pr[3])

11 Января 2016 года в 8 утра придет 9.0 ,в 9 утра 18.0 ,в 10 утра 26.0 ,в 11 утра 29.0


In [19]:
model.save_model("bst.model")
